In [1]:
import h5py
import os
import numpy as np
import matplotlib.pyplot as plt
from celluloid import Camera
import matplotlib.ticker as ticker
import pandas as pd

In [2]:
def fmt(x, pos):
    a, b = '{:.2e}'.format(x).split('e')
    b = int(b)
    return r'${} \times 10^{{{}}}$'.format(a, b)

In [3]:
gconst = 6.67430e-8
clight = 2.99792458e10
solar = 1.98847e33
lencgs2code = (clight**2)/(solar*gconst)
masscgs2code = (1.0e0/solar)
rhocgs2code = (masscgs2code/lencgs2code**3)
tcgs2code = (clight**3)/(solar*gconst)
energycgs2code = (1.0E0/clight**2)
me2 = 9.1093837015e-28*masscgs2code
mb2 = 1.66053906660e-24*masscgs2code
ye = 0.5e0
h_bar = (1.054571817e-27)*(lencgs2code**2*masscgs2code/tcgs2code)
amax = (me2**4)/(2.4e1*np.pi**2*h_bar**3)
bmax = (mb2*me2**3)/(3.0e0*np.pi**2*h_bar**3*ye)

In [4]:
def xe(rho):
    return (rho/bmax)**(1.0e0/3.0e0)
def dpdx(x):
    return 8.0e0*x**4/np.sqrt(x**2 + 1.0e0)
def cs(rho):
    return np.sqrt(amax*dpdx(xe(rho))/3.0e0/(rho**2*bmax)**(1.0e0/3.0e0))

In [5]:
filename = "/home/cnchong/Codes/cumc3d/model/Type_Ia/outfile/rkiter-0-nm.hdf5"
f = h5py.File(filename, "r")
f.keys()

<KeysViewHDF5 ['bfield', 'coordinate', 'dimension', 'epsilon', 'primitive', 'time', 'x-interface', 'y-interface', 'z-interface']>

In [6]:
# Face value
x1f = np.loadtxt('/home/cnchong/Codes/cumc3d/model/Type_Ia/profile/hydro_x1_fgrid.dat').T
x2f = np.loadtxt('/home/cnchong/Codes/cumc3d/model/Type_Ia/profile/hydro_x2_fgrid.dat').T
# Center value
x1f = x1f [3:-3] # Remove BC
x1c = (x1f[1:]+x1f[:-1])/2 #mid pt
x2f = x2f[3:-3] # Remove BC
x2c = (x2f[1:]+x2f[:-1])/2 #mid pt

rr, tt = np.meshgrid(x1c,x2c,indexing='ij') # r and cos, spherical
yy = rr*np.cos(tt)
xx = rr*np.sin(tt)

rrf, ttf = np.meshgrid(x1f,x2f,indexing='ij') # r and cos, spherical
yyf = rrf*np.cos(ttf)
xxf = rrf*np.sin(ttf)

In [7]:
# Initial Values
irho = np.loadtxt('/home/cnchong/Codes/cumc3d/model/Type_Ia/profile/hydro_rho.dat').T*rhocgs2code
ieps = np.loadtxt('/home/cnchong/Codes/cumc3d/model/Type_Ia/profile/hydro_eps.dat').T*energycgs2code
ivz = np.loadtxt('/home/cnchong/Codes/cumc3d/model/Type_Ia/profile/hydro_vphi.dat').T
iHe4 = np.loadtxt('/home/cnchong/Codes/cumc3d/model/Type_Ia/profile/Helm_Xhe4.dat').T
iC12 = np.loadtxt('/home/cnchong/Codes/cumc3d/model/Type_Ia/profile/Helm_Xc12.dat').T
iO16 = np.loadtxt('/home/cnchong/Codes/cumc3d/model/Type_Ia/profile/Helm_Xo16.dat').T
iNe20 = np.loadtxt('/home/cnchong/Codes/cumc3d/model/Type_Ia/profile/Helm_Xne20.dat').T
iMg24 = np.loadtxt('/home/cnchong/Codes/cumc3d/model/Type_Ia/profile/Helm_Xmg24.dat').T
iSi28 = np.loadtxt('/home/cnchong/Codes/cumc3d/model/Type_Ia/profile/Helm_Xsi28.dat').T
iNi56 = np.loadtxt('/home/cnchong/Codes/cumc3d/model/Type_Ia/profile/Helm_Xni56.dat').T

In [8]:
# Free fall time compared to simulation time
sim_time = 134863/tcgs2code
ff_time = np.sqrt((3*np.pi)/(32*gconst*np.average(irho)/rhocgs2code))
print(sim_time, ff_time)

0.6642865730192676 0.024800737574159987


In [9]:
data_list = []
n = 11
for i in range(n):
    filename = "/home/cnchong/Codes/cumc3d/model/Type_Ia/outfile/rkiter-"+str(i)+"-nm.hdf5"
    f = h5py.File(filename, "r")
    primitive = f['primitive'][:]
    primitive = primitive.T
    data_list.append(primitive)
    f.close()
epsilon_list = []
for i in range(n):
    filename = "/home/cnchong/Codes/cumc3d/model/Type_Ia/outfile/rkiter-"+str(i)+"-nm.hdf5"
    f = h5py.File(filename, "r")
    epsilon = f['epsilon'][:]
    epsilon = epsilon.T
    epsilon_list.append(epsilon)
    f.close()

In [10]:
name = ['rho', 'vx', 'vy', 'vz']
ioname = ['He4', 'C12', 'O16', 'Ne20', 'Mg24', 'Si28', 'Ni56']
quan_name = name+ioname+['q']
dict = {
    'rho' : irho,
    'vx' : 0,
    'vy' : 0,
    'vz' : ivz,
    'He4' : iHe4,
    'C12' : iC12,
    'O16' : iO16,
    'Ne20' : iNe20,
    'Mg24' : iMg24,
    'Si28' : iSi28,
    'Ni56' : iNi56,
    'q' : 1.0e-10
}
def easy_animation(xxf,yyf,data_list, name, ani_name):
    data_list_arr = np.array(data_list)
    fig = plt.figure()
    camera = Camera(fig)
    ax = plt.axes()
    vmin = np.min(data_list_arr)
    vmax = np.max(data_list_arr)
    print(vmin)
    print(vmax)
    for i in range(len(data_list)):
        prim = data_list[i]
        quan = prim
        plt.pcolormesh(xxf,yyf,quan,vmin=vmin,vmax=vmax)
        ax.text(0.3, 1.01, str(name)+ ' at time step ' +str(i), transform=ax.transAxes)
        camera.snap()
    plt.colorbar(format=ticker.FuncFormatter(fmt))
    plt.annotate('max = '+np.format_float_scientific(vmax, precision=3, exp_digits=1), xy=(0.05, 0.95), xycoords='axes fraction', color = 'red')
    plt.annotate('min = '+np.format_float_scientific(vmin, precision=3, exp_digits=1), xy=(0.05, 0.9), xycoords='axes fraction', color = 'red')
    ani = camera.animate()
    ani.save('/home/cnchong/Codes/cumc3d/model/Type_Ia/outfile/'+str(ani_name)+'.mp4')
    plt.close()
    

In [13]:
eps_anilist = []
for i in range(len(epsilon_list)):
    eps_anilist.append(epsilon_list[i][1:-1,1:-1,1])
easy_animation(xxf,yyf,np.log(np.array(eps_anilist)), 'log_eps', 'log_eps')

-11.691023231539535
-5.736758607413528


In [12]:
rho_list = []
q_list = []
for i in range(len(data_list)):
    rho = data_list[i][0,1:-1,1:-1,1]
    q = data_list[i][5,1:-1,1:-1,1]
    rho_list.append(rho)
    q_list.append(q)
easy_animation(xxf,yyf,np.log(np.array(rho_list)),'logrho', 'logrho')
easy_animation(xxf,yyf,np.array(q_list),'turbq', 'turbq')


-36.35936052672609
-17.93867978277373
0.0
0.0
